In [1]:
import numpy as np
import yfinance as yf
import scipy.stats as stats
import pandas as pd
from math import pi

In [2]:
# symbol = 'EURUSD=X'

# # Fetch the historical data from 2014 to 2024
# data = yf.download(symbol, start='2014-01-01', end='2025-01-01', interval='1d')

In [3]:
data = pd.read_csv('usd_eur_ts.csv')
data.set_index('Date', inplace=True)
data.drop('Log_Returns', axis=1, inplace=True)

In [4]:
# data.to_csv('usd_eur_ts.csv', index=True)

In [5]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,1.376595,1.377467,1.363271,1.376671,1.376671,0
2014-01-03,1.366624,1.367297,1.360170,1.366662,1.366662,0
2014-01-06,1.359582,1.364610,1.357279,1.359601,1.359601,0
2014-01-07,1.363066,1.365799,1.359878,1.363196,1.363196,0
2014-01-08,1.361693,1.363605,1.357202,1.361637,1.361637,0
...,...,...,...,...,...,...
2024-05-20,1.087548,1.088495,1.085505,1.087548,1.087548,0
2024-05-21,1.086083,1.087548,1.084599,1.086083,1.086083,0
2024-05-22,1.085517,1.086484,1.082251,1.085517,1.085517,0


In [6]:
S_0 =  data.iloc[-1]["Adj Close"]
S_0

1.0854227542877195

In [7]:
data['Log_Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))

# Drop NaN values that appear due to shift
data = data.dropna()

# Calculate the daily volatility (standard deviation of daily returns)
daily_volatility = data['Log_Returns'].std(ddof=1)

# Annualize the daily volatility
annualized_volatility = daily_volatility * np.sqrt(252)

In [8]:
annualized_volatility, daily_volatility

(0.07842553465367798, 0.004940344312640714)

In [9]:
r_d = 5.209/100
r_f = 3.67/100

# d1 and d2 for C1
d1 = (np.log(1/0.9) + (r_d - r_f + annualized_volatility**2/2))/annualized_volatility
d2 = d1 - annualized_volatility

In [10]:
r_f

0.036699999999999997

In [11]:
d1, d2

(1.5788964454943226, 1.5004709108406447)

In [12]:
# Calculate N(d1) and N(d2) for C1
N_d1 = stats.norm.cdf(d1)
N_d2 = stats.norm.cdf(d2)

print(f"N(d1) = {N_d1}")
print(f"N(d2) = {N_d2}")

N(d1) = 0.9428200942719626
N(d2) = 0.9332537684327362


In [13]:
# price of C1
S_0*np.exp(-r_f) * N_d1 - S_0*0.9*np.exp(-r_d)*N_d2

0.121078180283412

In [14]:
# d1 and d2 for C2

d1_c2 = (np.log(1/1.1) + (r_d - r_f + annualized_volatility**2/2))/annualized_volatility
d2_c2 = d1_c2 - annualized_volatility
d1_c2, d2_c2

(-0.9798453769019148, -1.0582709115555928)

In [15]:
# Calculate N(d1) and N(d2) for C2

N_d1_c2 = stats.norm.cdf(d1_c2)
N_d2_c2 = stats.norm.cdf(d2_c2)

print(f"N(d1) = {N_d1_c2}")
print(f"N(d2) = {N_d2_c2}")

N(d1) = 0.16358122466714464
N(d2) = 0.14496597524150734


In [16]:
# price of C2
S_0*np.exp(-r_f) * N_d1_c2 - S_0*1.1*np.exp(-r_d)*N_d2_c2

0.006857507313118266

In [17]:
# delta C1
delta_c1 = np.exp(-r_f)*N_d1
delta_c1

0.9088458376347525

In [18]:
# delta C2
delta_c2 = np.exp(-r_f)*N_d1_c2
delta_c2

0.15768662129409902

In [19]:
# gamma of C1
gamma_C1= np.exp(-r_f) * (1/(2*pi)**0.5) * np.exp(-d1**2/2)*(1/annualized_volatility)*(1/S_0)
gamma_C1

1.2989297226982202

In [20]:
# gamma of C2
gamma_C2= np.exp(-r_f) * (1/(2*pi)**0.5) * np.exp(-d1_c2**2/2)*(1/annualized_volatility)*(1/S_0)
gamma_C2

2.795327145994627

In [21]:
k = gamma_C2/gamma_C1
k

2.1520233906019133

In [22]:
n_2_w = 1/(k+1-k*delta_c1 + delta_c2)
n_2_w

0.7386328953724729

In [23]:
n_1_w = n_2_w*k
n_1_w

1.5895552679095775

In [24]:
n_3_w = (k*delta_c1-delta_c2)*(-n_2_w)
n_3_w

-1.3281881632820502

In [25]:
n_1_w+n_2_w + n_3_w

1.0000000000000002

In [26]:
CAPITAL = 100000

In [27]:
# Expected return for portfolio 

CAPITAL*np.exp(0.05209)

105347.05505964908

In [28]:
# Theta for C1
theta_1 = 0.5*S_0*np.exp(-r_f)*(1/(2*pi)**0.5)*np.exp(-d1**2/2) * annualized_volatility * 1
theta_1 -= r_f*S_0*np.exp(-r_f)*N_d1
theta_1 += r_d*S_0*0.9*np.exp(-r_d)*N_d2
theta_1

0.013581174235755306

In [29]:
# Theta for C2
theta_2 = 0.5*S_0*np.exp(-r_f)*(1/(2*pi)**0.5)*np.exp(-d1_c2**2/2) * annualized_volatility * 1
theta_2 -= r_f*S_0*np.exp(-r_f)*N_d1_c2
theta_2 += r_d*S_0*1.1*np.exp(-r_d)*N_d2_c2
theta_2

0.012404701388958474

In [30]:
# theta of portfolio

N1 = 126689.292
N2 = 58826.85804

N1*theta_1 - N2*theta_2

990.8597408196299